## Import models from Hugging Face hub
This sample shows how to import and register models from [HuggingFace hub](https://huggingface.co/models). 

### How does import work?
The import process runs as a job in your AzureML workspace using components from the `azureml` system registry. The models are downloaded and converted to MLflow packaged format. You can then register the models to your AzureML Workspace or Registry that makes them available for inference or fine tuning. 

### What models are supported for import?
Any model from Hugging Face hub can be downloaded using the `download_model` component. Only the following set of tasks are supported for MLflow conversion:
* fill-mask
* token-classification
* question-answering
* summarization
* text-generation
* text-classification
* translation
* image-classification
* text-to-image
Conversion to MLflow will fail if you attempt to download a model that has a task type other than the above.

### Why convert to MLflow?
MLflow is AzureML's recommended model packaging format. 
* **Inference benefits**: AzureML supports no-code-deployment for models packaged as MLflow that enables a seamless inference experience for the models. Learn more about [MLflow model packaging](https://learn.microsoft.com/en-us/azure/machine-learning/concept-mlflow-models) and [no-code-deployment](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-mlflow-models-online-endpoints?tabs=sdk). 
* **Fine tuning benefits**: Foundation models imported and converted to MLflow format can be fine tuned using AzureML's fine tuning pipelines. You can use the no-code UI wizards, or the code based job submission with the SDK or CLI/YAML. AzureML's fine tuning pipelines are built using components. This gives you the flexibility to compose your own fine tuning pipelines containing your own jobs for data transformation, post processing and the AzureML fine tuning components. Learn more about pipelines using [sdk](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-component-pipeline-python) or [CLI](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-component-pipelines-cli).

### What happens if I just download model and register models without converting to MLflow? That's because the task of the model I'm interested in is not among the supported list of tasks.
You can still download and register the model using the outputs of the `download_model` job. You need to [write your own inference code](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-online-endpoints?tabs=python) in this case. It also means that fine tuning is not yet supported if the task type of the model you are interested in is not in the supported list.

### Outline
* Setup pre-requisites such as compute.
* Pick a model to import.
* Configure the import job.
* Run the fine tuning job.
* Register the fine tuned model. 


### 1. Setup pre-requisites
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `azureml` system registry
* Set an optional experiment name
* Check or create compute.

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, ClientSecretCredential
from azure.ai.ml.entities import AmlCompute
import time

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

workspace_ml_client = MLClient(
        credential,
        subscription_id =  "<SUBSCRIPTION_ID>",
        resource_group_name =  "<RESOURCE_GROUP>",
        workspace_name =  "<WORKSPACE_NAME>"
)

# the models, fine tuning pipelines and environments are available in the AzureML system registry, "azureml-preview"
registry_ml_client = MLClient(credential, registry_name="azureml-preview-test1")

experiment_name = "question-answering-extractive-qna"

# If you already have a gpu cluster, mention it here. Else will create a new one with the name 'gpu-cluster-big'
compute_cluster = "gpu-cluster-big"
try:
    workspace_ml_client.compute.get(compute_cluster)
except Exception as ex:
    compute = AmlCompute(
        name = compute_cluster, 
        size= "Standard_ND40rs_v2",
        max_instances= 2 # For multi node training set this to an integer value more than 1
    )
    workspace_ml_client.compute.begin_create_or_update(compute).wait()

# This is the number of GPUs in a single node of the selected 'vm_size' compute. 
# Setting this to less than the number of GPUs will result in underutilized GPUs, taking longer to train.
# Setting this to more than the number of GPUs will result in an error.
gpus_per_node = 2 

# genrating a unique timestamp that can be used for names and versions that need to be unique
timestamp = str(int(time.time())) 


### 2. Pick a model to import
Browse models on [HuggingFace hub](https://huggingface.co/models) and identify a model to import. Make sure the task type of the model is among the supported tasks as explained in the introduction section. Copy the model id which is available in the URI of the page or can be copied using the copy icon next to the model name and assign it to the variable `huggingface_model_id`.

In [ ]:
huggingface_model_id = "microsoft/deberta-base"

### 3. Configure the model import job

In [ ]:
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities import CommandComponent, PipelineComponent, Job, Component
from azure.ai.ml import PyTorchDistribution, Input
from azure.ai.ml import load_component

# fetch the download_model component from the system registry
download_model_func = registry_ml_client.components.get(name="download_model", version="0.0.2")
# fetch the mlflow converter component from the system registry
convert_model_to_mlflow_func = registry_ml_client.components.get(name="convert_model_to_mlflow", version="0.0.1")
# fetch the model registration component - cannot use system registry version since it needs managed identity.
# we are investigating how to use obo
register_model_func = load_component(source="./tools/register-model/register.yml")

# define the pipeline job
@pipeline()
def create_pipeline():

    download_model_job = download_model_func(
        model_id = huggingface_model_id,
        model_source = "Huggingface"
    )
    convert_model_to_mlflow_job = convert_model_to_mlflow_func(
        model_download_details = download_model_job.outputs.model_info,
        model_path = download_model_job.outputs.model_output,
        # we will do away with the need to specify task_type soon
        task_type = "fill-mask"
    )

# blocked on Bug 2317665: how to use obo in pipeline dsl?
#    register_model_job = register_model_func(
#        model_path = convert_model_to_mlflow_job.outputs.mlflow_model_folder,
#        download_details = download_model_job.outputs.model_info,
#    )
#    register_model_job.identity.type = "user_identity"

    return {
        "downloaded_model": download_model_job.outputs.model_output,
        "downloaded_model_mlflow": convert_model_to_mlflow_job.outputs.mlflow_model_folder
    }

pipeline_object = create_pipeline()

# don't use cached results from previous jobs
pipeline_object.settings.force_rerun = True
pipeline_object.settings.default_compute  = compute_cluster

### 4. Submit the import job

In [ ]:
# submit the pipeline job
pipeline_job = workspace_ml_client.jobs.create_or_update(pipeline_object, experiment_name=experiment_name)
# wait for the pipeline job to complete
workspace_ml_client.jobs.stream(pipeline_job.name)

### 5. Register the downloaded model

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
# check if the `trained_model` output is available
print ("pipeline job outputs: ", workspace_ml_client.jobs.get(pipeline_job.name).outputs)

#fetch the model from pipeline job output - not working, hence fetching from fine tune child job
model_path_from_job = ("azureml://jobs/{0}/outputs/{1}".format(pipeline_job.name, "downloaded_model_mlflow"))

# replace any slashes in the model name with dashes because slash is not a valid character in for model names in AzureML
huggingface_model_id_registered = huggingface_model_id.replace("/", "-")

print("path to register model: ", model_path_from_job)
prepare_to_register_model = Model(
    path=model_path_from_job,
    type=AssetTypes.MLFLOW_MODEL,
    name=huggingface_model_id_registered,
    version=timestamp, # use timestamp as version to avoid version conflict
    description="Imported from Huggingface. View the model card at https://huggingface.co/" + huggingface_model_id"
)
print("prepare to register model: \n", prepare_to_register_model)
#register the model from pipeline job output 
registered_model = workspace_ml_client.models.create_or_update(prepare_to_register_model)
print ("registered model: \n", registered_model)
